In [1]:
library(terra)
library(sf)
library(tidyterra)
library(MultiscaleDTM)
library(randomForest)
library(caret)
library(vip)
library(rstac)
library(whitebox)
library(tmap)
library(stringr)


s_obj <- stac("https://planetarycomputer.microsoft.com/api/stac/v1")

terra 1.7.71

Linking to GEOS 3.10.2, GDAL 3.4.3, PROJ 8.2.0; sf_use_s2() is TRUE


Attaching package: ‘tidyterra’


The following object is masked from ‘package:stats’:

    filter


Warning message in rgl.init(initValue, onlyNULL):
“RGL: unable to open X11 display”
Warning message:
“'rgl.init' failed, running with 'rgl.useNULL = TRUE'.”
randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.

Loading required package: ggplot2


Attaching package: ‘ggplot2’


The following object is masked from ‘package:randomForest’:

    margin


Loading required package: lattice


Attaching package: ‘vip’


The following object is masked from ‘package:utils’:

    vi



Attaching package: ‘tmap’


The following object is masked from ‘package:datasets’:

    rivers




In [2]:
it_obj <- s_obj %>% 
    stac_search(collections = "sentinel-2-l2a",
                datetime = "2018-07-01T00:00:00Z/2018-08-30T00:00:00Z",
                limit = 10,
                bbox = c(-47.02148, -17.35063, -42.53906, -12.98314)) %>%
    get_request()

print(it_obj)

###STACItemCollection
- features (10 item(s)):
  - S2A_MSIL2A_20180828T132231_R038_T23LNF_20201027T054353
  - S2A_MSIL2A_20180828T132231_R038_T23LNE_20201027T054349
  - S2A_MSIL2A_20180828T132231_R038_T23LND_20201027T054349
  - S2A_MSIL2A_20180828T132231_R038_T23LMF_20201027T054359
  - S2A_MSIL2A_20180828T132231_R038_T23LME_20201027T054358
  - S2A_MSIL2A_20180828T132231_R038_T23LMD_20201027T054354
  - S2A_MSIL2A_20180828T132231_R038_T23LMC_20201027T054355
  - S2A_MSIL2A_20180828T132231_R038_T23LLF_20201027T054355
  - S2A_MSIL2A_20180828T132231_R038_T23LLE_20201027T054356
  - S2A_MSIL2A_20180828T132231_R038_T23LLD_20201027T054357
- assets: 
AOT, B01, B02, B03, B04, B05, B06, B07, B08, B09, B11, B12, B8A, datastrip-metadata, granule-metadata, inspire-metadata, preview, product-metadata, rendered_preview, safe-manifest, SCL, tilejson, visual, WVP
- item's fields: 
assets, bbox, collection, geometry, id, links, properties, stac_extensions, stac_version, type


In [3]:
# list_files <- list.files(path = "data/Spectral/", full.names = TRUE, pattern = ".tif", include.dirs = FALSE)
# list_files[[2]]

In [4]:
# for(i in 1:length(list_files)){
#     terra::project(rast(list_files[[i]]), "EPSG:3740", 
#                    filename = paste0(str_sub(list_files[[i]], 1, 25), i, "_prj3740.tif"))
# }

In [5]:
list_files <- list.files(path = "data/Spectral/", full.names = TRUE, pattern = "3740.tif", include.dirs = FALSE)
list_files

[1] "data/Spectral//WA_median_1_prj3740.tif"
[2] "data/Spectral//WA_median_2_prj3740.tif"
[3] "data/Spectral//WA_median_3_prj3740.tif"
[4] "data/Spectral//WA_median_4_prj3740.tif"
[5] "data/Spectral//WA_median_5_prj3740.tif"
[6] "data/Spectral//WA_median_6_prj3740.tif"

In [11]:
terraOptions(memmin = 1, memmax =NA)

In [12]:
terraOptions()

memfrac   : 0.6
tolerance : 0.1
verbose   : FALSE
todisk    : FALSE
tempdir   : /tmp/RtmpjS1XFn
datatype  : FLT4S
memmin    : 1
progress  : 3


In [15]:
system.time(rast(list_files[[1]])+rast(list_files[[1]]))

   user  system elapsed 
 83.183   7.479  93.271 

In [16]:
terraOptions(memmin = 0.01, memmax =0.05)

In [17]:
terraOptions()

memfrac   : 0.6
tolerance : 0.1
verbose   : FALSE
memmax    : 0.05
todisk    : FALSE
tempdir   : /tmp/RtmpjS1XFn
datatype  : FLT4S
memmin    : 0.01
progress  : 3


In [18]:
system.time(rast(list_files[[1]])+rast(list_files[[1]]))

   user  system elapsed 
 81.254   2.475  86.407 

In [22]:
wa_spec_sprc <- sprc(lapply(list_files, rast))

In [23]:
wa_spec_sprc

class       : SpatRasterCollection 
length      : 6 
nrow        : 8960, 8960, 8960, 4406, 4406, 4406 
ncol        : 8960, 8960, 2366, 8960, 8960, 2366 
nlyr        :    7,    7,    7,    7,    7,    7 
extent      : 362550, 971130, 5043570, 5444550  (xmin, xmax, ymin, ymax)
crs (first) : NAD83(HARN) / UTM zone 10N (EPSG:3740) 

In [26]:
wa_spec_merge <- terra::merge(wa_spec_sprc, filename = "data/Spectral/WA_median_mosaic.tif")